<a href="https://colab.research.google.com/github/abrarrhine/Text_Classification/blob/main/BERTopic_with_restaurant_review_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 61.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
import os
import pandas as pd
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/CS5934:Capstone/datasets/Restaurant_Reviews.tsv', sep='\t')

In [ ]:
df.Liked.value_counts()

1    500
0    500
Name: Liked, dtype: int64

In [ ]:
df['len_chac'] = df.Review.str.len()
df.len_chac.describe()

count    1000.000000
mean       58.315000
std        32.360052
min        11.000000
25%        33.000000
50%        51.000000
75%        80.000000
max       149.000000
Name: len_chac, dtype: float64

In [ ]:
docs = df.Review.to_list()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
%%time
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model_embedding.encode(docs)

CPU times: user 3.11 s, sys: 1.22 s, total: 4.33 s
Wall time: 36.3 s


In [ ]:
%%time
model = BERTopic(
    n_gram_range=(1, 2),
    vectorizer_model=vectorizer_model,
    nr_topics='auto',
    min_topic_size=10,
    seed_topic_list=[
        ["experience", "bad", "good", "nice"],
        ["place", "atmosphere", "toilet", "clean"],
        ["staff", "waitress", "service"],
        ["wait", "time", "long"],
        ["food", "taste"]
    ],
    calculate_probabilities=True).fit(docs, corpus_embeddings)

CPU times: user 5.62 s, sys: 148 ms, total: 5.77 s
Wall time: 4.47 s


In [ ]:
topics, probabilities = model.transform(docs, corpus_embeddings)

In [ ]:
df_topic_freq = model.get_topic_freq()
topics_count = len(df_topic_freq) - 1
df_topic_freq

,Topic,Count
0,-1,304
1,0,123
2,1,106
3,2,56
4,3,55
5,4,47
6,5,46
7,6,37
8,7,34
9,8,30


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart(top_n_topics=topics_count)

In [ ]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(corpus_embeddings)
model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)